In [139]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import talib as ta
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

coin_name = "AAVE"
coin_pair = coin_name + "-USD"
ticker = yf.Ticker( coin_pair )
the_data = ticker.history(period='60d',interval="15m")


In [140]:
# Adding indicators
data=the_data.copy()
data.columns = data.columns.get_level_values(0)
data['RSI_15'] = ta.RSI(data.Close, timeperiod=15)
data['RSI_7'] = ta.RSI(data.Close, timeperiod=7)
data['EMAF'] = ta.EMA(data.Close, timeperiod=20)
data['EMAM'] = ta.EMA(data.Close, timeperiod=100)
data['EMAS'] = ta.EMA(data.Close, timeperiod=150)
data['ADX'] = ta.ADX(data['High'], data['Low'], data['Close'])

In [141]:
data.dropna(inplace=True)
data.reset_index(inplace = True)
data.drop(['Datetime','Volume','Dividends','Stock Splits'], axis=1, inplace=True)
data = data.loc[:, ['Close','High','Low','Open','RSI_15','RSI_7','EMAF','EMAM','EMAS','ADX']]
data_set = data.iloc[:, 0:10]#.values
pd.set_option('display.max_columns', None)
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)

In [142]:
backcandles = 30
last_window = data_set_scaled[-backcandles:, :]
print( last_window.shape )
X_input = np.expand_dims(last_window, axis=0)

(30, 10)


In [143]:
model = load_model('/data/oracle/' + coin_pair + '.keras')

In [145]:
y_pred = model.predict(X_input)
#y_pred_original = sc.inverse_transform(np.concatenate([np.zeros((y_pred.shape[0], data_set_scaled.shape[1] - 1)), y_pred], axis=1))[:, -1]

y_pred_adjusted = np.concatenate([y_pred, np.zeros((y_pred.shape[0], data_set_scaled.shape[1] - 1))], axis=1)
y_pred_original = sc.inverse_transform(y_pred_adjusted)[:, 0]

print(y_pred_original[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
336.13963099429384
